In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
import catboost
import shap

In [8]:
data=pd.read_csv("df_all.csv")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_23852\106750930.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("df_all.csv")


In [9]:
data.head()

,Unnamed: 0,"Полож.пед.акселер.,%","Нагрузка на двигатель, %","Давл.масла двиг.,кПа","Темп.масла двиг.,°С","Обор.двиг.,об/мин","Значение счетчика моточасов, час:мин",Сост.пед.сцепл.,iButton2,КПП. Температура масла,...,"Крутящий момент (spn513), Нм","Положение рейки ТНВД (spn51), %","Расход топлива (spn183), л/ч","ДВС. Температура наддувочного воздуха, °С","Давление наддувочного воздуха двигателя (spn106), кПа",Текущая передача (spn523),"Температура масла гидравлики (spn5536), С",Педаль слива (spn598),problem,anomaly
0,384,0.0,NaN,0.0,NaN,0.000,NaN,Отпущ.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,386,0.0,NaN,380.0,-273.0,649.000,25344.0,Отпущ.,NaN,-273.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,387,0.0,NaN,360.0,-273.0,651.000,25344.0,Отпущ.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,388,0.0,NaN,348.0,-273.0,656.000,25344.0,Отпущ.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,389,0.0,NaN,360.0,-273.0,671.875,25344.0,Отпущ.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [10]:
num_feat = ['Полож.пед.акселер.,%', 'Нагрузка на двигатель, %',
       'Давл.масла двиг.,кПа', 'Обор.двиг.,об/мин',
       'Значение счетчика моточасов, час:мин', 
        'КПП. Давление масла в системе смазки',
       'Скорость', 'ДВС. Давление смазки',
       'ДВС. Температура охлаждающей жидкости',
       'Давление в пневмостистеме (spn46), кПа', 'Уровень топлива % (spn96)',
        'ДВС. Частота вращения коленчатого вала']
features = num_feat

# Разделение на признаки и целевую переменную
X = data[features]
def pp(x):
    if x["problem"]==1:
        return 2
    if x["anomaly"]==1:
        return 1
    return 0
y = data.apply(pp,axis=1)
# X.loc[:,cat_feat]= X[cat_feat].fillna('no_data')

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=y,test_size=0.2)

In [12]:
X

,"Полож.пед.акселер.,%","Нагрузка на двигатель, %","Давл.масла двиг.,кПа","Обор.двиг.,об/мин","Значение счетчика моточасов, час:мин",КПП. Давление масла в системе смазки,Скорость,ДВС. Давление смазки,ДВС. Температура охлаждающей жидкости,"Давление в пневмостистеме (spn46), кПа",Уровень топлива % (spn96),ДВС. Частота вращения коленчатого вала
0,0.0,NaN,0.0,0.000,NaN,NaN,0.0,0.0,35.0,NaN,NaN,0.000
1,0.0,NaN,380.0,649.000,25344.0,1184.0,0.0,380.0,34.0,760.0,NaN,649.000
2,0.0,NaN,360.0,651.000,25344.0,1200.0,0.0,360.0,34.0,816.0,NaN,651.000
3,0.0,NaN,348.0,656.000,25344.0,1200.0,0.0,348.0,35.0,816.0,NaN,656.000
4,0.0,NaN,360.0,671.875,25344.0,1200.0,0.0,360.0,36.0,816.0,NaN,671.875
...,...,...,...,...,...,...,...,...,...,...,...,...
2856638,8.0,NaN,458.0,3350.000,10116.0,2839.0,15.0,290.0,122.0,417.0,NaN,125.000
2856639,82.0,NaN,941.0,2778.000,10116.0,1895.0,20.0,1492.0,44.0,279.0,NaN,1350.000
2856640,10.0,NaN,1187.0,2518.000,10116.0,34.0,22.0,762.0,96.0,773.0,NaN,1867.000
2856641,22.0,NaN,1199.0,1291.000,10116.0,1468.0,25.0,486.0,114.0,324.0,NaN,818.000


In [15]:
from catboost import CatBoostClassifier
cbc=CatBoostClassifier(task_type="GPU").fit(X_train,y_train)

Learning rate set to 0.314419
0:	learn: 0.6615099	total: 43.1ms	remaining: 43.1s
1:	learn: 0.4760783	total: 61.4ms	remaining: 30.7s
2:	learn: 0.3681034	total: 79.1ms	remaining: 26.3s
3:	learn: 0.2953169	total: 96.6ms	remaining: 24.1s
4:	learn: 0.2499629	total: 112ms	remaining: 22.3s
5:	learn: 0.2065503	total: 130ms	remaining: 21.6s
6:	learn: 0.1814144	total: 148ms	remaining: 20.9s
7:	learn: 0.1576881	total: 166ms	remaining: 20.6s
8:	learn: 0.1426485	total: 183ms	remaining: 20.2s
9:	learn: 0.1315947	total: 199ms	remaining: 19.7s
10:	learn: 0.1216949	total: 215ms	remaining: 19.4s
11:	learn: 0.1136239	total: 232ms	remaining: 19.1s
12:	learn: 0.1070364	total: 248ms	remaining: 18.8s
13:	learn: 0.1008578	total: 267ms	remaining: 18.8s
14:	learn: 0.0955783	total: 285ms	remaining: 18.7s
15:	learn: 0.0929219	total: 300ms	remaining: 18.5s
16:	learn: 0.0885786	total: 318ms	remaining: 18.4s
17:	learn: 0.0858300	total: 333ms	remaining: 18.2s
18:	learn: 0.0827757	total: 349ms	remaining: 18s
19:	learn

In [16]:
pred=cbc.predict(X_test)

In [17]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99    102977
           1       1.00      1.00      1.00     37941
           2       1.00      1.00      1.00    430411

    accuracy                           1.00    571329
   macro avg       1.00      1.00      1.00    571329
weighted avg       1.00      1.00      1.00    571329



In [18]:
cbc.fit(X,y)

Learning rate set to 0.329062
0:	learn: 0.6457521	total: 18.8ms	remaining: 18.7s
1:	learn: 0.4605373	total: 38.3ms	remaining: 19.1s
2:	learn: 0.3542712	total: 55.6ms	remaining: 18.5s
3:	learn: 0.2827872	total: 74.2ms	remaining: 18.5s
4:	learn: 0.2390590	total: 92.7ms	remaining: 18.4s
5:	learn: 0.1969121	total: 114ms	remaining: 18.9s
6:	learn: 0.1698034	total: 134ms	remaining: 19.1s
7:	learn: 0.1526138	total: 152ms	remaining: 18.8s
8:	learn: 0.1414676	total: 168ms	remaining: 18.5s
9:	learn: 0.1310719	total: 185ms	remaining: 18.4s
10:	learn: 0.1189802	total: 204ms	remaining: 18.3s
11:	learn: 0.1109823	total: 223ms	remaining: 18.3s
12:	learn: 0.1056339	total: 240ms	remaining: 18.2s
13:	learn: 0.0988615	total: 259ms	remaining: 18.3s
14:	learn: 0.0942068	total: 279ms	remaining: 18.3s
15:	learn: 0.0889847	total: 297ms	remaining: 18.3s
16:	learn: 0.0869256	total: 314ms	remaining: 18.2s
17:	learn: 0.0825428	total: 334ms	remaining: 18.2s
18:	learn: 0.0788792	total: 354ms	remaining: 18.3s
19:	le

In [23]:
import joblib
joblib.dump(cbc,"main_model.pkl")

['main_model.pkl']

In [24]:
df=data.loc[data["problem"]+data["anomaly"]==0,num_feat].mean(axis=0)

In [25]:
df.to_csv("normal_stats.csv")

In [26]:
d=['Полож.пед.акселер.,%', 'Нагрузка на двигатель, %',
       'Давл.масла двиг.,кПа', 'Обор.двиг.,об/мин',
       'Значение счетчика моточасов, час:мин',
        'КПП. Давление масла в системе смазки',
       'Скорость', 'ДВС. Давление смазки',
       'ДВС. Температура охлаждающей жидкости',
       'Давление в пневмостистеме (spn46), кПа', 'Уровень топлива % (spn96)',
        'ДВС. Частота вращения коленчатого вала']
for i in d:
    print(i,i in df.index)

Полож.пед.акселер.,% True
Нагрузка на двигатель, % True
Давл.масла двиг.,кПа True
Обор.двиг.,об/мин True
Значение счетчика моточасов, час:мин True
КПП. Давление масла в системе смазки True
Скорость True
ДВС. Давление смазки True
ДВС. Температура охлаждающей жидкости True
Давление в пневмостистеме (spn46), кПа True
Уровень топлива % (spn96) True
ДВС. Частота вращения коленчатого вала True
